<a href="https://colab.research.google.com/github/eturcec23/Analiza_uspjesnosti_videoigara/blob/main/Analiza_uspje%C5%A1nosti_videoigara_Ela_Tur%C4%8Dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analiza uspješnosti videoigara

**Student:** Ela Turčec\
**Kolegij:** Programiranje za analizu podataka (PzAP)\
**Akademska godina:** 2025./2026.

---

## 1. Uvod i motivacija

Glavna motivacija za odabir ove teme proizlazi iz mog dugogodišnjeg interesa za područje videoigara. Kao osoba koja igra igrice cijeli život pratim trendove u industriji, često primjećujem diskrepanciju između onoga što marketing obećava i stvarnog iskustva igranja. Često svjedočimo naslovima s ogromnim budžetima koji unatoč "hypeu" razočaraju zajednicu, dok s druge strane manje "indie" igre postaju globalni hitovi isključivo zbog svoje kvalitete.

Potaknuta tim iskustvom, odlučila sam iskoristiti alate inženjerstva podataka kako bih istražila što zapravo čini igru uspješnom.

**Cilj ovog projekta** je analiza uspješnosti videoigara kroz integraciju triju ključnih dimenzija:
1.  **Komercijalni uspjeh:** Globalna prodaja (izražena u milijunima primjeraka).
2.  **Kvaliteta:** Korisničke i kritičke ocjene (preuzete s relevantnih servisa).
3.  **Financijska učinkovitost:** Procjena povrata ulaganja (ROI) u odnosu na žanr i godinu izlaska.

S tehničke strane, projekt demonstrira cjelokupni "data pipeline" (ETL proces): od prikupljanja heterogenih podataka (CSV, API), preko njihovog čišćenja i obogaćivanja sintetičkim podacima (generator), do pohrane u bazu i izlaganja rezultata putem REST API-ja.

In [1]:
!pip install flask sqlalchemy

import pandas as pd
import numpy as np
import requests
import time
import threading
import sqlite3
from sqlalchemy import create_engine, text
from flask import Flask, jsonify, request
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Opis implementacije

Sustav je razvijen u jeziku **Python** unutar okruženja Google Colab. Arhitektura rješenja oslanja se na modularni pristup, gdje svaka komponenta obavlja specifičan dio obrade podataka.

### 2.1. Prikupljanje podataka
Koriste se tri heterogena izvora podataka:
* **Strukturirani podaci (CSV):** Osnovni skup podataka preuzet je s platforme Kaggle ("Video Game Sales Dataset") i sadrži povijesne podatke o prodaji igara po platformama. Za učitavanje i manipulaciju koristi se biblioteka `pandas` [1].
* **Nestrukturirani podaci (JSON/REST API):** Za svaku igru iz osnovnog skupa dohvaćaju se dodatni metapodaci (trenutna ocjena, prosječno vrijeme igranja, datum izlaska) s vanjskog servisa **RAWG Video Games Database API** [2]. Komunikacija se odvija putem HTTP GET zahtjeva koristeći biblioteku `requests`.
* **Generirani podaci (Synthetic Data):** S obzirom na to da su točni razvojni budžeti poslovna tajna, implementiran je **Python generator** (koristeći biblioteku `numpy` [3]). Generator procjenjuje budžet na temelju žanra i godine izlaska, dodajući stohastički šum (random noise) kako bi simulirao realne varijacije u troškovima razvoja.

In [6]:
# 1. Spajanje na Google Drive
drive.mount('/content/drive', force_remount=True)

# 2. Učitavanje CSV-a
putanja = '/content/drive/MyDrive/pzap projekt/Video_Games_Sales_as_at_22_Dec_2016.csv'

try:
    df = pd.read_csv(putanja)
    df = df[['Name', 'Platform', 'Year_of_Release', 'Genre', 'Global_Sales']].dropna()
    display(df.head(3))
except FileNotFoundError:
    print("Greška: Provjeri putanju do datoteke!")

Mounted at /content/drive


,Name,Platform,Year_of_Release,Genre,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,82.53
1,Super Mario Bros.,NES,1985.0,Platform,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,35.52


### Dohvat podataka s vanjskog servisa (API)

U nastavku se vrši povezivanje na **RAWG Video Games Database API**. Zbog ograničenja broja zahtjeva u besplatnoj verziji API-ja (Rate Limiting) te radi bržeg izvođenja demonstracije, dohvat podataka se vrši na reprezentativnom uzorku od prvih 15 najprodavanijih igara.

Za svaku igru dohvaćamo:
* Trenutnu ocjenu (*Rating*)
* Prosječno vrijeme igranja (*Playtime*)
* Točan datum izlaska

In [10]:
# Moj RAWG ključ
API_KEY = "2215ce5b23a54c9ab17025a40f6c3efa"

# Uzorak od prvih 15 igara
subset_games = df.head(15).copy()
integrated_data = []

print("Povezujem se na RAWG API...")

for index, row in subset_games.iterrows():
    game_name = str(row['Name']).split('(')[0].strip()
    url = f"https://api.rawg.io/api/games?key={API_KEY}&search={game_name}"

    try:
        response = requests.get(url)
        data = response.json()

        # Ako API pronađe igru, uzimamo podatke
        if data.get('results'):
            api_game = data['results'][0]

            new_record = {
                'Naslov': row['Name'],
                'Platforma': row['Platform'],
                'Prodaja_Milijuni': row['Global_Sales'],
                'Godina': row['Year_of_Release'],
                'Zanr': row['Genre'],
                # Novi podaci s API-ja
                'RAWG_Ocjena': api_game.get('rating', 0),
                'Vrijeme_Igranja': api_game.get('playtime', 0),
                'Datum_Izlaska_API': api_game.get('released', 'N/A')
            }
            integrated_data.append(new_record)
            print(f"Dohvaćeno: {game_name}")

        time.sleep(0.2)

    except Exception as e:
        print(f"Greška kod {game_name}: {e}")

# glavna tablica
df_final = pd.DataFrame(integrated_data)
print("\n--- INTEGRIRANI PODACI ---")
display(df_final.head(3))

Povezujem se na RAWG API...
Dohvaćeno: Wii Sports
Dohvaćeno: Super Mario Bros.
Dohvaćeno: Mario Kart Wii
Dohvaćeno: Wii Sports Resort
Dohvaćeno: Pokemon Red/Pokemon Blue
Dohvaćeno: Tetris
Dohvaćeno: New Super Mario Bros.
Dohvaćeno: Wii Play
Dohvaćeno: New Super Mario Bros. Wii
Dohvaćeno: Duck Hunt
Dohvaćeno: Nintendogs
Dohvaćeno: Mario Kart DS
Dohvaćeno: Pokemon Gold/Pokemon Silver
Dohvaćeno: Wii Fit
Dohvaćeno: Kinect Adventures!

--- INTEGRIRANI PODACI ---


,Naslov,Platforma,Prodaja_Milijuni,Godina,Zanr,RAWG_Ocjena,Vrijeme_Igranja,Datum_Izlaska_API
0,Wii Sports,Wii,82.53,2006.0,Sports,4.21,55,2006-11-19
1,Super Mario Bros.,NES,40.24,1985.0,Platform,4.32,30,1985-09-13
2,Mario Kart Wii,Wii,35.52,2008.0,Racing,4.30,12,2008-04-27
